In [7]:
# Setup the Jupyter version of Dash
from jupyter_dash import JupyterDash

# Dashboard imports
import dash_leaflet as dl
from dash import dcc, html, dash_table
from dash.dependencies import Input, Output
import plotly.express as px
import base64

# Data tools
import pandas as pd

# Import YOUR CRUD module
from CRUD_Python_Module import AnimalShelter

# ---------------------------
# Database / Model
# ---------------------------
username = "aacuser"
password = "SNHU1234"

db = AnimalShelter(username, password)

df = pd.DataFrame.from_records(db.read({}))
df.drop(columns=["_id"], inplace=True)

# ---------------------------
# App Setup
# ---------------------------
app = JupyterDash(__name__)
JupyterDash.infer_jupyter_proxy_config()

# Logo (optional but required for rubric)
image_filename = 'Grazioso Salvare Logo.png'
encoded_image = base64.b64encode(open(image_filename, 'rb').read())

# ---------------------------
# Layout
# ---------------------------
app.layout = html.Div([
    html.Center(html.H1("CS-340 Grazioso Salvare Dashboard")),
    html.Center(html.H4("Robert Chappell")),

    html.Center(
        html.Img(
            src='data:image/png;base64,{}'.format(encoded_image.decode()),
            style={'height': '120px'}
        )
    ),

    html.Hr(),

    # FILTER CONTROLS
    dcc.RadioItems(
        id='filter-type',
        options=[
            {'label': 'Reset (All)', 'value': 'RESET'},
            {'label': 'Water Rescue', 'value': 'WATER'},
            {'label': 'Mountain / Wilderness Rescue', 'value': 'MOUNTAIN'},
            {'label': 'Disaster / Individual Tracking', 'value': 'DISASTER'}
        ],
        value='RESET',
        inline=True
    ),

    html.Hr(),

    # DATA TABLE
    dash_table.DataTable(
        id='datatable-id',
        columns=[{"name": i, "id": i} for i in df.columns],
        data=df.to_dict('records'),
        page_size=10,
        sort_action="native",
        filter_action="native",
        row_selectable="single",
        selected_rows=[0],
        style_table={'overflowX': 'auto'}
    ),

    html.Hr(),

    # CHART + MAP
    html.Div(style={'display': 'flex'}, children=[
        html.Div(id='graph-id', style={'width': '45%'}),
        html.Div(id='map-id', style={'width': '55%'})
    ])
])

# ---------------------------
# Filter Callback
# ---------------------------
@app.callback(
    Output('datatable-id', 'data'),
    Input('filter-type', 'value')
)
def update_dashboard(filter_type):

    WATER_BREEDS = ["Labrador Retriever", "Newfoundland", "Chesapeake Bay Retriever"]
    MOUNTAIN_BREEDS = ["German Shepherd", "Alaskan Malamute", "Siberian Husky", "Rottweiler"]
    DISASTER_BREEDS = ["Doberman Pinscher", "Bloodhound", "Golden Retriever"]

    base_age = {"age_upon_outcome_in_weeks": {"$lte": 104}}

    if filter_type == "WATER":
        query = {**base_age, "breed": {"$in": WATER_BREEDS}}
    elif filter_type == "MOUNTAIN":
        query = {**base_age, "breed": {"$in": MOUNTAIN_BREEDS}}
    elif filter_type == "DISASTER":
        query = {**base_age, "breed": {"$in": DISASTER_BREEDS}}
    else:
        query = {}

    dff = pd.DataFrame.from_records(db.read(query))

    if dff.empty:
        return df.to_dict('records')

    if "_id" in dff.columns:
        dff.drop(columns=["_id"], inplace=True)

    return dff.to_dict('records')





    return dff.to_dict('records')

# ---------------------------
# Chart Callback
# ---------------------------
@app.callback(
    Output('graph-id', 'children'),
    Input('datatable-id', 'derived_virtual_data')
)
def update_chart(viewData):

    if viewData is None:
        return []

    dff = pd.DataFrame.from_dict(viewData)

    if "breed" not in dff.columns:
        return []

    counts = dff["breed"].value_counts().head(10).reset_index()
    counts.columns = ["breed", "count"]

    fig = px.bar(counts, x="breed", y="count", title="Top Breeds (Filtered View)")
 
    return [dcc.Graph(figure=fig)]

# ---------------------------
# Map Callback
# ---------------------------
@app.callback(
    Output('map-id', 'children'),
    Input('datatable-id', 'derived_virtual_data'),
    Input('datatable-id', 'derived_virtual_selected_rows')
)
def update_map(viewData, index):

    if viewData is None or index is None or len(index) == 0:
        return []

    dff = pd.DataFrame.from_dict(viewData)
    row = index[0]

    lat = dff.iloc[row]["location_lat"]
    lon = dff.iloc[row]["location_long"]
    breed = dff.iloc[row]["breed"]
    name = dff.iloc[row]["name"]

    return [
        dl.Map(style={'width': '100%', 'height': '500px'}, center=[30.75, -97.48], zoom=10, children=[
            dl.TileLayer(),
            dl.Marker(position=[lat, lon], children=[
                dl.Tooltip(breed),
                dl.Popup([html.B("Animal Name: "), html.Span(name)])
            ])
        ])
    ]

# ---------------------------
# Run App
# ---------------------------
app.run_server()



Dash app running on https://oasishand-podiumruby-3000.codio.io/proxy/8050/
